In [2]:
import os

In [3]:
import faiss

In [4]:
import numpy as np

In [5]:
import re

In [6]:
from sentence_transformers import SentenceTransformer
EMBEDDING_MODEL = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

C:\Users\aravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [7]:
from langchain.document_loaders import PDFPlumberLoader

In [8]:
from langchain_ollama.llms import OllamaLLM 

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [31]:
# EMBEDDING_MODEL = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
EMBEDDING_MODEL = SentenceTransformer("all-MiniLM-L12-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\aravi\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aravi\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
LANGUAGE_MODEL = OllamaLLM(model="deepseek-r1:1.5b")

In [12]:
PDF_STORAGE_PATH = "C:\\Users\\aravi\\Downloads\\pdfwrite"

In [13]:
dimension = EMBEDDING_MODEL.get_sentence_embedding_dimension()
DOCUMENT_VECTOR_DB = faiss.IndexFlatL2(dimension)

In [14]:
doc_store = {}

In [15]:
def save_uploaded_file(uploaded_file, filename):
    """Saves uploaded PDF to a local directory."""
    file_path = os.path.join(PDF_STORAGE_PATH, filename)
    with open(file_path, "wb") as file:
        file.write(uploaded_file.read())
    return file_path

In [16]:
def load_pdf_documents(file_path):
    """Loads documents from a PDF file."""
    document_loader = PDFPlumberLoader(file_path)
    return document_loader.load()

In [17]:
def chunk_documents(raw_documents):
    """Splits documents into smaller text chunks."""
    text_processor = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=300,
        add_start_index=True
    )
    return text_processor.split_documents(raw_documents)

In [18]:
def index_documents(document_chunks):
    """Embeds documents, adds them to FAISS, and stores text mappings."""
    embeddings = np.array(EMBEDDING_MODEL.encode(
        [doc.page_content for doc in document_chunks], 
        convert_to_numpy=True, 
        normalize_embeddings=True
    ), dtype=np.float32)

    start_id = len(doc_store)  # Start index for new documents
    for i, doc in enumerate(document_chunks):
        doc_store[start_id + i] = doc.page_content  # Store text
    
    DOCUMENT_VECTOR_DB.add(embeddings) 

In [19]:
def find_related_documents(query, k=2):
    """Finds the most relevant documents using FAISS similarity search."""
    query_embedding = np.array(EMBEDDING_MODEL.encode(
        [query], 
        convert_to_numpy=True, 
        normalize_embeddings=True
    ), dtype=np.float32)

    _, indices = DOCUMENT_VECTOR_DB.search(query_embedding, k)

    results = [doc_store[idx] for idx in indices[0] if idx in doc_store]
    return results  

In [20]:
# def expand_query(query):
#     """Expands a short query into a more detailed search phrase."""
#     expansion_prompt = f"""
#     Given this short user query: "{query}"
#     Expand it into a more detailed question for better document retrieval.
#     """
#     return LANGUAGE_MODEL.invoke(expansion_prompt)

def prompt_template(user_query, context_text):
    """Formats user query and document context into a structured prompt."""
    return f"""You are an intelligent assistant tasked with answering user queries strictly based on the provided document.

### **Context (Extracted from Document)**
{context_text}

### **User Query**
{user_query}

### **Instructions for Answering**
- Use only the provided document content to generate a response.
- Do **not** introduce any external information or assumptions.
- Extract relevant details from the document and structure them into a **coherent, well-formed response**.
- Ensure the response is **clear and readable**, maintaining natural formatting without unnecessary line breaks (`\n`).
- Do **not** insert extra newlines unless they are essential for meaning or readability.
- Avoid excessive spacing, unnatural paragraph breaks, or disjointed formatting.
- If special characters like `\n` are found in the extracted content, **omit them unless needed for clarity**.
- If no relevant information is available, respond with: "I'm sorry, but I couldn't find an answer in the provided document."
- Do not provide explanations or context beyond what is necessary to answer the query.

### **Final Answer (Ensure clear, natural formatting without unnecessary newlines)**
"""



In [21]:
def generate_answer(user_query,related_docs):
    """Finds related documents, formats a prompt, and generates an AI response."""
    

    if not related_docs:
        return ["I'm sorry, but I couldn't find an answer in the provided document.", ""]

    context_text = "\n\n".join([doc for doc in related_docs])
    prompt = prompt_template(user_query, context_text)
    ai_response1 = LANGUAGE_MODEL.invoke(prompt)
    ai_response2 = LANGUAGE_MODEL.invoke(f"response from an AI model {ai_response1} refine the response to clear,concise and meaninglful by without changing or avoiding any information ")
    ai_response1 = re.split(r"</think>\s*", ai_response1, maxsplit=1)[-1].strip()
    ai_response2 = re.split(r"</think>\s*", ai_response2, maxsplit=1)[-1].strip()
    return [ai_response1,ai_response2, context_text]


In [22]:
uploaded_pdf = "C:\\Users\\aravi\\Downloads\\mypdf.pdf"
if uploaded_pdf:
    filename = os.path.basename(uploaded_pdf)
    file_path = save_uploaded_file(open(uploaded_pdf, "rb"), filename)
    print(f"PDF saved at: {file_path}")

PDF saved at: C:\Users\aravi\Downloads\pdfwrite\mypdf.pdf


In [23]:
# Load and process document
raw_docs = load_pdf_documents(file_path)


In [24]:
processed_chunks = chunk_documents(raw_docs)

In [25]:
index_documents(processed_chunks)

In [26]:
# User query example
user_input = "How to switch from Airtel Prepaid to Postpaid?"
related_docs = find_related_documents(user_input)

In [27]:
ai_response = generate_answer(user_input,related_docs)

In [43]:
ai_response[0]

'To switch from Airtel Prepaid to Postpaid:\n\n1. Go to Airtel\'s website and click on the "Switch prepaid to postpaid page" option under the Prepaid dropdown menu.\n\n2. Fill out all your details as required by the form.\n\n3. Ensure you have a valid Prepaid number from another operator with no contractual obligations.\n\n4. Your current operator\'s mobile number must be at least 90 days old.\n\n5. Post verification will occur within 1-2 business days, converting your plan to postpaid.'

In [29]:
ai_response[1]

'To switch your Airtel mobile plan to SMS:\n\n1. Visit Airtel Mobile\'s website and select "Switch mobile prepaid to SMS page."\n\n2. Fill out all details as required by the form.\n\n3. Ensure you have a valid SMS number from another provider, with no contractual obligations.\n\n4. The current operator\'s SMS number must be at least 90 days old for porting from your previous SIM.\n\n5. Post verification will process within 1-2 business days, converting your plan to SMS.'